## ReLU

* Sigmoid의 문제점
    * 역전파 단계 : Gradient계산
    * 그래프 양끝의 sigmoid의 미분값이 아주 작아지고 소멸되는 현상이 발생(Vanishing Gradient)
    * `torch.nn.sigmoid(x)`

* ReLU : $f(x) = max(0,x)$ 0보다 크면 x을 출력, 0보다 작으면 0을 출력 
    * 음수일 경우에는 마찬가지로 Vanishing Gradient가 발생하지만 양수 일 경우 문제 없음
    * `torch.nn.relu(x)`
    
* 그 외의 Activation
    * `torch.nn.tanh(x)`
    * `torch.nn.leaky_relu(x,0.01)` : relu에서 음의 영역을 0값으로 넣지 않고 변형
  
* torch.optim에서 Optimizer를 사용가능 
    * SGD
    * Adam 등등

* Optimizer : 오솔길을 찾는 방법에 비유 
    * GD : 모든 자료를 다 검토, 내 위치의 산 기울기를 계산해서 방향을 찾음
    * SGD : 전부 다 봐야 너무 오래 걸리니까 조금만 보고 빨리 판단. 같은 시간에 더 많이 감
    * Adagrad : 안가본 곳은 성큼 빠르게 걷고, 많이 가본 곳은 잘 아니까 갈수록 보폭을 줄여 세밀히 탐색
    * Momentum : 스텝 계산해서 움직인 후, 아까 내려오던 관성 방향 또 감
    * RMSProp : 보폭을 줄이면서 맥락 상황까지 고려
    * Adadelta : 보폭이 너무 작아져서 정지하는 것을 방지하자
    * Adam : RMSProp + Momentum 
 

In [12]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [14]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [15]:
# torchvision.datasets 패키지 사용
mnist_train = dsets.MNIST(root='MNIST_data/', # 데이터를 어디에 저장을 할지 
                          train=True,         # train set을 불러올지 
                          transform=transforms.ToTensor(), # tensor 
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,          # train set이 아님 : test set
                         transform=transforms.ToTensor(),
                         download=True)

In [16]:
# dataset loader 
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,   # mnist_train set
                                          batch_size=batch_size, # 배치사이즈 100
                                          shuffle=True,   # 섞을지 
                                          drop_last=True) # 마지막 배치를 버릴지 말지 

In [17]:
# MNIST data image of shape 28 * 28 = 784
linear1 = torch.nn.Linear(784, 256, bias=True).to(device)
linear2 = torch.nn.Linear(256, 256, bias=True).to(device)
linear3 = torch.nn.Linear(256, 10, bias=True).to(device)
relu = torch.nn.ReLU()

In [18]:
# Initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [19]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [20]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 130.154266357
Epoch: 0002 cost = 36.310234070
Epoch: 0003 cost = 22.885091782
Epoch: 0004 cost = 15.922073364
Epoch: 0005 cost = 11.547133446
Epoch: 0006 cost = 8.626523972
Epoch: 0007 cost = 6.448483944
Epoch: 0008 cost = 4.804335117
Epoch: 0009 cost = 3.676800728
Epoch: 0010 cost = 2.644905806
Epoch: 0011 cost = 1.990795493
Epoch: 0012 cost = 1.596786976
Epoch: 0013 cost = 1.220444679
Epoch: 0014 cost = 0.993460000
Epoch: 0015 cost = 0.814860165
Learning finished


In [22]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9470000267028809
Label:  8
Prediction:  5
